Python script to retrieve the PnL of open margen possitions on Binance.
Right now the script only works for USDT pairs.

In [20]:
# https://python-binance.readthedocs.io/en/latest/margin.html#id12
try:
  from binance.client import Client
  from dateutil.parser import parse
except Exception as e:
  !pip install python-binance
  from binance.client import Client

In [21]:
import time
import pandas as pd
import numpy as np

In [22]:
key = 'YourKey'
sec = 'YourSecret'

In [23]:
client = Client(key, sec)

In [24]:
info = client.get_margin_account()

In [25]:
holdings = [i for i in info['userAssets'] if float(i['netAsset']) > 0]

In [26]:
all_symbols = [i['asset'] for i in info['userAssets']]
#  try to get pnl for all possitions open and closed 

In [ ]:
assets = [i['asset'] for i in holdings]
assets

In [28]:
# The trades pairs
quote_currency = 'USDT'

In [29]:
# trades = client.get_margin_trades(symbol = f'ADA{quote_currency}') 
# trades
#  'isBuyer': True, = a buy
#  'isBuyer': False, = a sell

In [30]:
def get_entry_order(trades, current_amount):
  # Find the entry order so the first buy order for the current possition
  x = current_amount
  entry_order_id = None

  for i in trades[::-1]: # reverse the list
    if i['isBuyer']:
      x -= float(i['qty']) 
    elif i['isBuyer'] is False:
      x += float(i['qty'])
    if x <= 0.01:   
      entry_order_id = i['id']
      # print(f"Entry Order ID: {entry_order_id}")
      return entry_order_id

In [31]:
def slice_trades_to_entry_order(trades, entry_order_id):
  # Find the index of the entry order for current possition in the trades list
  for i, v in enumerate(trades):
      if int(v['id']) == int(entry_order_id):
        return trades[i:]

In [32]:
prices = client.get_all_tickers()
# prices

In [ ]:
my_finance = []
for asset in assets:
  print('Asset:', asset)

  trades = client.get_margin_trades(symbol = f'{asset}{quote_currency if asset != "BTC" else "USDC"}') 
  total_Amount = float([i['netAsset'] for i in holdings if i['asset'] == asset][0])
  # Filter out tiny amounts  
  if float([x for x in prices if x['symbol'] == asset + quote_currency][0]['price']) * total_Amount < 5:
    print(f'Skipp {asset} total_Amount to small')
    continue
  entry_order_id = get_entry_order(trades, total_Amount)
  if not entry_order_id:
    # print(trades)
    continue
  trades_ = slice_trades_to_entry_order(trades, entry_order_id)
  cost = 0

  for i in trades_:
    if i['isBuyer']:
      # total_Amount += float(i['qty'])
      cost += float(i['qty']) * float(i['price'])
    elif i['isBuyer'] is False:
      # total_Amount -= float(i['qty'])
      cost -= float(i['qty']) * float(i['price'])

  print(f'Asset: {asset}, Current Amount: {total_Amount}, Total Spend: {cost}, Average Entry Price: {cost / total_Amount}')
  # Average Cost per coin = Total spend ÷ total number of coins owned
  my_finance.append({'asset':asset, 'amount':total_Amount, 'cost':cost, 'avg_entry_price': cost / total_Amount})
  time.sleep(1)

print()
pd.DataFrame(my_finance)

In [ ]:
pnl_total = 0
pnl_ = []
date = pd.Timestamp('now')
for i in my_finance:
  price = [x for x in prices if x['symbol'] == i['asset'] + quote_currency][0]['price']
  pnl = (float(price) - i['avg_entry_price']) /  i['avg_entry_price']
  pnl_total += pnl
  #pnl_.append({'asset':i['asset'], 'pnl %':pnl, 'pnl $': i['cost'] * pnl]})
  pnl_.append({
      'date': date,
      'asset': i['asset'], 
      'pnl %': pnl, 
      'pnl $': i['cost'] * pnl,
      'invested': i['cost'],
      'avg_entry_price': i['avg_entry_price'],
      'last_price' : price
      })
  print(f'Asset:{i["asset"]}, PnL: {pnl}')

# print()
# print(f'Total PnL: {pnl_}')

In [ ]:
df = pd.DataFrame(pnl_).round(3)
df['pnl %'] = df['pnl %'] * 100
df['pct_holding'] = ( df.invested / df.invested.sum()).round(3) * 100
df.sort_values(by='pnl %',ascending=False, inplace=True)
df.set_index(df.date, inplace=True)
df.drop(columns='date',inplace=True)
df

In [36]:
total_invested_usd = df['invested'].sum()
# total_invested_usd

total_borrowed = [x for x in info['userAssets'] if x['borrowed'] != '0']
borrowed_assets = [x for x in total_borrowed if x['asset'] != 'USDT']
convert_usd = [float(x['borrowed']) * float([y['price'] for y  in prices if y['symbol'] == x['asset'] + 'USDT'][0]) for x in borrowed_assets]

total_borrowed_usd = sum([float(x['borrowed']) for x in total_borrowed if x['asset'] == 'USDT'] + convert_usd)
# total_borrowed_usd

In [ ]:
df2 = pd.DataFrame(index=[1])
df2['total_invested_usd'] = total_invested_usd
df2['total_borrowed_usd'] = total_borrowed_usd
df2['borow_ratio'] = df2['total_borrowed_usd'] / df2['total_invested_usd']
df2['total_pnl_usd'] = df['pnl $'].sum()
df2.round(2)